In [1]:
#Libraries
import numpy as np
import pandas as pd 
import csv
import pprint
import pickle
from datetime import datetime
import time
import json
import os

In [2]:
#create a time stamp, Month_Day_HourMinuteSeconds
time_stamp=time.strftime('%B_%d_%H%M%S')
dir_name='./data_parser_output/'+time_stamp
os.makedirs(dir_name,exist_ok=False)

In [3]:
def split_data(data_df, as_dic=False):
    full_state_trace = np.array(data_df[state_names])
    full_action_trace = np.array(data_df[action_names])
    actions_input=np.array(data_df[action_names].iloc[0:len(data_df)-1,:]) #skip last row
    states_input=np.array(data_df[state_names].iloc[0:len(data_df)-1,:]) #skip last row
    states_output=np.array(data_df[state_names].iloc[1::,:]) #skip fist row
    states_diff = states_output-states_input
    diff_out=pd.DataFrame(data=states_diff,columns=state_names)
    print("Actions input shape:",actions_input.shape)
    print("State input shape:",states_input.shape)
    print("State output shape:",states_output.shape)
    print("State difference shape:",states_diff.shape)
    print("Diff_out df shape:",diff_out.shape)
    print("full_action_trace",full_action_trace.shape)
    print("full_state_trace:",full_state_trace.shape)
    data_dic={'actions_input':actions_input, 
                'states_input':states_input,
                'states_output':states_output,
                'states_diff':states_diff,
                'diff_out': diff_out,
                'full_action_trace':full_action_trace,
                'full_state_trace':full_state_trace
                    }
    if as_dic:
        print('Return dictionary keys: \n',data_dic.keys())
        return data_dic
    print('Return a list with: [actions_input, states_input,states_output,states_diff, diff_out,full_action_trace,full_state_trace]')
    return actions_input, states_input,states_output,states_diff, diff_out,full_action_trace,full_state_trace

In [4]:
def shift_data(input_ds,output_ds,shift):
    in_ds_shift=input_ds[0:len(input_ds)-shift,:]
    out_ds_shift=output_ds[shift::,:]
    print("Shift is:%s \n Input Shape:%s \n Output Shape:%s"%(shift,in_ds_shift.shape,out_ds_shift.shape))
    return in_ds_shift,out_ds_shift

In [5]:
def get_diff(actions,states,shift,verbose=False):
    a_in=actions[0:len(states)-shift,:]
    s_in=states[0:len(states)-shift,:]
    s_out=states[shift::,:]
    s_diff=s_out-s_in
    a_s_in=np.concatenate((a_in,s_in),axis=1)
    if verbose:
        print('original_input\n',states.shape)
        print('states cut\n',s_in.shape)
        print('states shift\n',s_out.shape)
        print('diff\n',s_diff.shape)
        print('action_state_input\n',a_s_in.shape)
    return s_in,s_out,s_diff,a_s_in  

In [6]:
def get_shift_cut(actions,states,shift):
    _,num_actions_columns=actions.shape
    a_s_in=np.concatenate((actions,states),axis=1)
    print('a_s_shift\n',a_s_in)
    r,c=a_s_in.shape #rows, columns
    n=r//shift
    a_s_shift=np.zeros((n,c))
    #print(a_s_shift.shape)
    #print(n)
    for i in range(1,n):
        #print(i)
        a_s_shift[i,:]=a_s_in[i*shift,:]
    x_set=a_s_shift[0:-1,:]
    y_set=a_s_shift[1::,num_actions_columns::]
    return a_s_shift,x_set,y_set

In [7]:
#loading Data in a Data Frame
data_dir=r'C:\Users\aalbaher\pttgc\data_1585254368.csv'
data_df=pd.read_csv(data_dir)
#You may be able to load the x_set and y_set directly and skip the split of the data if needed.

In [8]:
#Asuming the states_names and action names where store in env_data directory
state_names = np.load('./env_data/state_names.npy', allow_pickle=True)
action_names = np.load('./env_data/action_names.npy', allow_pickle=True)

## Split Data

In [9]:
actions_input, states_input,states_output,states_diff, diff_out,full_action_trace,full_state_trace = split_data(data_df=data_df)

Actions input shape: (494624, 8)
State input shape: (494624, 30)
State output shape: (494624, 30)
State difference shape: (494624, 30)
Diff_out df shape: (494624, 30)
full_action_trace (494625, 8)
full_state_trace: (494625, 30)
Return a list with: [actions_input, states_input,states_output,states_diff, diff_out,full_action_trace,full_state_trace]


In [10]:
#data set for predict value of state
x_set_total = np.concatenate((actions_input,states_input),axis=1)
y_set_total = states_output
#data set for predict value difference between states (one step)
x_set_diff=x_set_total
y_set_diff=states_output-states_input
print(x_set_diff.shape)
print(y_set_diff.shape)

(494624, 38)
(494624, 30)


## Shift and get difference

In [11]:
#select shift step
shift=10

In [12]:
#shift data
s_in_shift,s_out_shift,s_diff,a_s_in_shift=get_diff(actions=actions_input,states=states_input,shift=shift,verbose=True)

original_input
 (494624, 30)
states cut
 (494614, 30)
states shift
 (494614, 30)
diff
 (494614, 30)
action_state_input
 (494614, 38)


In [13]:
x_name=('./data_parser_output/'+time_stamp+'/x_set_diff_s_%s.pickle'%(shift))
y_name=('./data_parser_output/'+time_stamp+'/y_set_diff_s_%s.pickle'%(shift))
print(x_name)
print(y_name)

./data_parser_output/April_27_184109/x_set_diff_s_10.pickle
./data_parser_output/April_27_184109/y_set_diff_s_10.pickle


In [14]:
#Diff with shift, writing dataset into disk
with open(x_name, 'wb') as f:
    pickle.dump(a_s_in_shift, f, pickle.HIGHEST_PROTOCOL)
with open(y_name, 'wb') as f:
    pickle.dump(s_diff, f, pickle.HIGHEST_PROTOCOL)
print(a_s_in_shift.shape)
print(s_diff.shape)

(494614, 38)
(494614, 30)


## Shift and Value

In [15]:
#Try with a smaller shift
shift = 5

In [16]:
#little validation
num_row,_=states_input.shape
print(num_row//shift-1)

98923


In [17]:
x_name_sv=('./data_parser_output/'+time_stamp+'/x_set_sv%s.pickle'%(shift))
y_name_sv=('./data_parser_output/'+time_stamp+'/y_set_sv%s.pickle'%(shift))
print(x_name_sv)
print(y_name_sv)

./data_parser_output/April_27_184109/x_set_sv5.pickle
./data_parser_output/April_27_184109/y_set_sv5.pickle


In [18]:
a_s_shift,x_set_sv,y_set_sv=get_shift_cut(actions=actions_input,states=states_input,shift=shift)

a_s_shift
 [[1998.030518   2000.000366    160.3250275  ...   82.73389435
     3.41233397   39.11010742]
 [2001.770142   2000.000977    160.3579559  ...   82.73149109
     3.40727234   39.27136993]
 [2003.749268   2000.001709    160.3852234  ...   83.05037689
     3.40911126   39.37528229]
 ...
 [2091.254639   2083.137207    164.9241791  ...   84.35935211
     3.80847549   37.7918663 ]
 [2088.333008   2083.137695    164.9267578  ...   84.36689758
     3.80079579   37.85303116]
 [2089.855469   2083.137939    164.9235077  ...   84.3693161
     3.77875304   37.76679611]]


In [19]:
print('x shape:',x_set_sv.shape)
print('y shape:',y_set_sv.shape)

x shape: (98923, 38)
y shape: (98923, 30)


In [20]:
with open(x_name_sv, 'wb') as f:
    pickle.dump(x_set_sv, f, pickle.HIGHEST_PROTOCOL)
with open(y_name_sv, 'wb') as f:
    pickle.dump(y_set_sv, f, pickle.HIGHEST_PROTOCOL)

## Create Subset

In [21]:
sub_set=10000
x_sub_diff=x_set_diff[0:sub_set]
y_sub_diff=y_set_diff[0:sub_set]
print(x_sub_diff.shape)
print(y_sub_diff.shape)

(10000, 38)
(10000, 30)


In [22]:
#writing dataset into disk
x_subset_filename = './data_parser_output/'+time_stamp+'/x_set_diff_subset.pickle'
y_subset_filename = './data_parser_output/'+time_stamp+'/y_set_diff_subset.pickle'
with open(x_subset_filename, 'wb') as f:
    pickle.dump(s_in_shift, f, pickle.HIGHEST_PROTOCOL)
with open(y_subset_filename, 'wb') as f:
    pickle.dump(s_out_shift, f, pickle.HIGHEST_PROTOCOL)

## Test and troubleshoot

In [23]:
#Check one step difference
print(states_input[0:11,0])
print(states_output[0:11,0])
assert(states_input[1,0]==states_output[0,0])

[6776.119141 6780.637695 6855.405762 6735.211914 6800.287598 6847.007813
 6840.42627  6776.625    6819.195801 6826.537598 6737.      ]
[6780.637695 6855.405762 6735.211914 6800.287598 6847.007813 6840.42627
 6776.625    6819.195801 6826.537598 6737.       6793.839844]


In [24]:
#check shift difference
print('shift',shift)
print(states_input[shift,0])
print(states_input[0,0])
diff_man=states_input[shift,0]-states_input[0,0]
#print(diff_man)
#print(s_diff[0,0])
assert(diff_man==s_diff[0,0])

shift 5
6847.007812999998
6776.119140999999


AssertionError: 